In [11]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 28.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.8.0
    Uninstalling google-auth-oauthlib-0.8.0:
      Successfully uninstalled google-auth-oauthlib-0.8.0
  Attempting uninstall: google-auth-httplib2
    Found existing installation: google-auth-httplib2 0.1.0
    Uninstalling google-auth-httplib2-0.1.0:
      Successfully uninstalled google-auth-httplib2-0.1.0
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.74.0
    Uninstalling google-api-python-client-2.74.0:
      Successfully uninstalled google-api-python-client-2.74.0


In [33]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 4.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.8/284.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00


In [66]:
from openai import OpenAI
import openai
import json

# Load the OpenAI API key from a JSON file
def load_api_key(filepath="openaikey.json"):
    with open(filepath, "r") as file:
        data = json.load(file)
        return data.get("plyhealthprotokey")
# Function to extract action items using OpenAI GPT model
def comprehend_email_with_gpt(body):
    api_key = load_api_key()
    client = OpenAI(api_key=api_key)
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "You are a helpful assistant helping healthcare providers respond to payer enrollment emails."},
        {
            "role": "user",
            "content": f"Extract actions from the following emails:\n{body}"
        }
      ]
    )
    return response

In [68]:
import os.path
import email
import base64

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

"""Shows basic usage of the Gmail API.
Lists the user's Gmail labels.
"""
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists("token.json"):
  creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
  if creds and creds.expired and creds.refresh_token:
    creds.refresh(Request())
  else:
    flow = InstalledAppFlow.from_client_secrets_file(
        "credentials.json", SCOPES
    )
    creds = flow.run_local_server(port=0)
  # Save the credentials for the next run
  with open("token.json", "w") as token:
    token.write(creds.to_json())

try:
  # Call the Gmail API
  service = build("gmail", "v1", credentials=creds)
  messages = service.users().messages().list(userId="me").execute().get("messages")
  print("Messages:")
  for message in messages:
    text = service.users().messages().get(userId="me", id=message['id'],format='raw').execute()
    # Decode the raw message from base64
    msg_str = base64.urlsafe_b64decode(text['raw'].encode('ASCII'))

    # Parse the message using the email library
    mime_msg = email.message_from_bytes(msg_str)

    # Walk through the parts to extract the body
    for part in mime_msg.walk():
        if part.get_content_type() == 'text/plain' and part.get_content_disposition() is None:
            body = part.get_payload(decode=True)  # Decode the message body
            body_text = body.decode()  # Convert from bytes to string
            response = comprehend_email_with_gpt(body_text)
            print(response.choices[0].message.content)

except HttpError as error:
  print(f"An error occurred: {error}")

Messages:
